In [ ]:
import numpy as np
from inputs.constants import INPUTS_PATH, SUPPLEMENT_PATH
from estival.model import BayesianCompartmentalModel
import arviz as az
from emutools.tex import StandardTexDoc
from autumn.core.runs import ManagedRun
from aust_covid.calibration import get_priors, get_targets
from aust_covid.model import build_model
from aust_covid.inputs import get_ifrs
from emutools.inputs import load_param_info
from estival.sampling import tools as esamp

In [ ]:
run_path = 'projects/aust_covid/base_case_analysis/2023-08-30T1608-try_DEMZ_10k'

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
mr = ManagedRun(run_path)
mr.remote.download(mr.remote.list_contents()[-1])
idata = az.from_netcdf(mr.list_local()[-1])
burnt_idata = idata.sel(draw=np.s_[2000:])

In [ ]:
sds = az.extract(burnt_idata, num_samples=20)

In [ ]:
param_info = load_param_info(INPUTS_PATH / 'parameters.yml')
param_info['value'].update(get_ifrs(app_doc))
parameters = param_info['value'].to_dict()
aust_model = build_model(app_doc)
priors = get_priors()
targets = get_targets(app_doc)
calibration_model = BayesianCompartmentalModel(aust_model, parameters, priors, targets)

In [ ]:
spaghetti_res = esamp.model_results_for_samples(sds, calibration_model)

In [ ]:
spaghetti_res.results.to_csv('spaghetti.csv')